In [1]:
from torch_geometric.datasets import Planetoid
dataset = Planetoid(root='./', name = 'cora')
g = dataset[0]
print(dataset.num_classes)
print(g)
print(g.keys)

7
Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])
['test_mask', 'train_mask', 'x', 'y', 'val_mask', 'edge_index']


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(g.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)
        self.relu = nn.ReLU()
        
    def forward(self, g):
        x, edge_index = g.x, g.edge_index
        
        x = self.conv1(x, edge_index)
        x = self.relu(x)
        x = F.dropout(x, training = self.training)
        x = self.conv2(x, edge_index)
        return x

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [4]:
model = GCN().to(device)
g = g.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01, weight_decay = 5e-4)
criterion = nn.CrossEntropyLoss()
print(model)

GCN(
  (conv1): GCNConv(1433, 16)
  (conv2): GCNConv(16, 7)
  (relu): ReLU()
)


In [5]:
for i in range(200):
    out = model(g)
    _, pred = out.max(1)
    print(pred)
    print(pred.shape)
    print(pred.sum().item())
    
    print(out.argmax(1))
    print(out.argmax(1).shape)
    print(out.argmax(1).sum().item())
    print(pred == out.argmax(1))
    break

tensor([2, 4, 4,  ..., 4, 4, 4])
torch.Size([2708])
8944
tensor([2, 4, 4,  ..., 4, 4, 4])
torch.Size([2708])
8944
tensor([True, True, True,  ..., True, True, True])


In [6]:
for epoch in range(200):
    model.train()
    
    out = model(g)
    loss = criterion(out[g.train_mask], g.y[g.train_mask])
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    pred = out.argmax(1)
    correct = (pred[g.train_mask] == g.y[g.train_mask]).sum().item()
    train_acc = correct / (g.train_mask.sum().item())
    
    if epoch % 10 == 0:
        print("Epoch {:05d}  /  Loss {:.4f}  /  Train_acc {:.4f}".
              format(epoch, loss.item(), train_acc))

Epoch 00000  /  Loss 1.9446  /  Train_acc 0.2143
Epoch 00010  /  Loss 0.7546  /  Train_acc 0.9500
Epoch 00020  /  Loss 0.2463  /  Train_acc 0.9857
Epoch 00030  /  Loss 0.0782  /  Train_acc 1.0000
Epoch 00040  /  Loss 0.0692  /  Train_acc 1.0000
Epoch 00050  /  Loss 0.0516  /  Train_acc 1.0000
Epoch 00060  /  Loss 0.0367  /  Train_acc 1.0000
Epoch 00070  /  Loss 0.0391  /  Train_acc 1.0000
Epoch 00080  /  Loss 0.0353  /  Train_acc 1.0000
Epoch 00090  /  Loss 0.0433  /  Train_acc 1.0000
Epoch 00100  /  Loss 0.0253  /  Train_acc 1.0000
Epoch 00110  /  Loss 0.0227  /  Train_acc 1.0000
Epoch 00120  /  Loss 0.0570  /  Train_acc 0.9929
Epoch 00130  /  Loss 0.0229  /  Train_acc 1.0000
Epoch 00140  /  Loss 0.0364  /  Train_acc 1.0000
Epoch 00150  /  Loss 0.0420  /  Train_acc 0.9929
Epoch 00160  /  Loss 0.0220  /  Train_acc 1.0000
Epoch 00170  /  Loss 0.0504  /  Train_acc 1.0000
Epoch 00180  /  Loss 0.0281  /  Train_acc 1.0000
Epoch 00190  /  Loss 0.0423  /  Train_acc 0.9929


In [7]:
model.eval()
out = model(g)
pred = out.argmax(1)
correct = (pred[g.test_mask] == g.y[g.test_mask]).sum().item()
test_acc = correct / (g.test_mask.sum().item())
print("Test Accuracy : {:.4f}".format(test_acc))

Test Accuracy : 0.7930
